(sec-tutorial)=
# Tutorial

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# Matplotlib customizations.
plt.rc("figure", dpi=300, figsize=(9,3))
plt.rc("font", family="serif")
plt.rc("legend", edgecolor='none')

The `rom_operator_inference` package constructs reduced-order models (ROM) for large dynamical systems.
Such systems often arise from the numerical solution of partial differentials equations (PDE).
In this tutorial, we give an example of solving a **very** simple heat equation with a ROM learned from data via Operator Inference.
This is a simplified version of the first numerical example of Operator Inference in the literature, described in {cite}`PW2016OperatorInference`.

## Problem Statement

Let $\Omega = [0,L]\subset \mathbb{R}$ be the spatial domain and let $[t_0,t_f]\subset\mathbb{R}$ be the time domain.
We consider the one-dimensional heat equation with homogeneous Dirichlet boundary conditions,

\begin{align*}
    \frac{\partial}{\partial t} q(x,t) - \frac{\partial^2}{\partial x^2}q(x,t) &= 0
    & x &\in\Omega,\quad t\in(t_0,t_f],
    \\
    q(0,t) = q(L,t) &= 0
    & t &\in [t_0,t_f],
    \\
    q(x,t_0) &= x(1 - x),
    & x &\in \Omega.
\end{align*}

This is a model for a one-dimensional rod that conducts heat: the temperature at the ends of the rod are fixed at $0$ and heat is allowed to flow out of the rod through the ends.

## Full-order Discretization

To solve the problem numerically, let $\{x\}_{i=0}^{n+1}$ be an equidistant grid of $n+2$ points on $\Omega$, i.e.,

\begin{align*}
    0 &= x_0 < x_1 < \cdots < x_n < x_{n+1} = L
    &
    &\text{and}
    &
    \delta x &= \frac{L}{n+1} = x_{i+1} - x_{i},\quad i=1,\ldots,n-1.
\end{align*}

Since the boundary conditions prescribe $q(x_0,t) = q(x_{n+1},t) = 0$, we wish to compute the state vector

\begin{align*}
    \mathbf{q}(t)
    = \left[\begin{array}{c}
        q(x_1,t) \\ \vdots \\ q(x_n,t)
    \end{array}\right]\in\mathbb{R}^n
\end{align*}

for $t\in[t_0,t_f]$.

Introducing the central finite difference approximation

\begin{align*}
    \frac{\partial^2}{\partial x^2}q(x,t) &\approx \frac{q(x-\delta x,t) - 2q(x,t) + q(x+\delta x,t)}{(\delta x)^2}
    % &
    % \Longrightarrow&
    % &
    % \frac{\partial^2}{\partial x^2}q(x_i,t) &\approx \frac{q(x_{i-1},t) - 2q(x_{i},t) + q(x_{i+1},t)}{(\delta x)^2}
    % \\
    &
    &\Longrightarrow
    &
    \frac{\partial^2}{\partial x^2}x_{i} &\approx \frac{x_{i-1} - 2x_{i} + x_{i+1}}{(\delta x)^2},
\end{align*}

we obtain the semi-discrete linear system

$$
\frac{\text{d}}{\text{d}t}\mathbf{q}(t) = \mathbf{A}\mathbf{q}(t),
\qquad
\mathbf{q}(0) = \mathbf{q}_0,
$$ (eq_fom)

where

\begin{align*}
    \mathbf{A} &= \frac{1}{(\delta x)^2}\left[\begin{array}{ccccc}
        -2 & 1 & & & \\
        1 & -2 & 1 & & \\
        & \ddots & \ddots & \ddots & \\
        & & 1 & -2 & 1 \\
        & & & 1 & -2 \\
    \end{array}\right] \in\mathbb{R}^{n\times n},
    &
    \mathbf{q}_0 &= \left[\begin{array}{c}
    x_1 (1 - x_1) \\ x_2 (1 - x_2) \\ \vdots \\ x_{n-1} (1 - x_{n-1}) \\ x_n (1 - x_n)
    \end{array}\right] \in\mathbb{R}^{n}.
\end{align*}

Equation {eq}`eq_fom` is called the _full-order model_ (FOM) or the _high-fidelity model_. The computational complexity of solving {eq}`eq_fom` depends on the large dimension $n$; our goal is to construct a reduced-order model (ROM) that approximates the full-order model, but whose computational complexity only depends on some smaller dimension $r \ll n$.

```{important}
One key advantage of Operator Inference is that, because it learns a reduced-order model from data alone, direct access to the high-fidelity solver (the matrix $\mathbf{A}$ in this case) is not required. In this tutorial, we explicitly construct the high-fidelity solver, but in practice, we only need the following:
1. Solution outputs of a high-fidelity solver to learn from, and
2. Some knowledge of the structure of the governing equations.
```

## Training Data Generation

For this demo, we set $t_0 = 0$ and $L = t_f = 1$.
We begin by simulating the full-order system described above with a maximal time step size $\delta t = 10^{-3}$, resulting in $k = 10^3+1$ time steps (1000 steps after the initial condition).
The results are organized as the _snapshot matrix_ $\mathbf{Q}\in\mathbb{R}^{n\times k}$, where the $j$th column is the solution trajectory at time $t_j$:

$$
    \mathbf{Q} = \left[\begin{array}{ccc}
        && \\
        \mathbf{q}_{0} & \cdots & \mathbf{q}_{k-1}
        \\ &&
    \end{array}\right] \in\mathbb{R}^{n\times k},
    \qquad
    \mathbf{q}_{j} := \mathbf{q}(t_j) \in\mathbb{R}^{n},\quad j = 0, \ldots, k-1.
$$

In [ ]:
# Construct the spatial domain.
L = 1                           # Spatial domain length.
n = 2**7 - 1                    # Spatial grid size.
x_all = np.linspace(0, L, n+2)  # Full spatial grid.
x = x_all[1:-1]                 # Interior spatial grid (where x is unknown).
dx = x[1] - x[0]                # Spatial resolution.

# Construct the temporal domain.
t0, tf = 0, 1                   # Initial and final time.
k = tf*1000 + 1                 # Temporal grid size.
t = np.linspace(t0, tf, k)      # Temporal grid.
dt = t[1] - t[0]                # Temporal resolution.

print(f"Spatial step size δx = {dx}")
print(f"Temporal step size δt = {dt}")

In [ ]:
# Construct the state matrix A.
diags = np.array([1,-2,1]) / (dx**2)
A = sparse.diags(diags, [-1,0,1], (n,n))

# Define the full-order model dx/dt = f(t,x),  x(0) = x0.
def fom(t, x):
    return A @ x

q0 = x * (1 - x)

print(f"shape of A:\t{A.shape}")
print(f"shape of q0:\t{q0.shape}")

In [ ]:
# Compute snapshots by solving the full-order model with SciPy.
Q = solve_ivp(fom, [t0,tf], q0, t_eval=t, method="BDF", max_step=dt).y

print(f"shape of Q: {Q.shape}")

```{caution}
It is often better to use your own ODE solver instead of integration packages such as `scipy.integrate`. If the integration strategy of the full-order model is known (e.g., Backward Euler), try using that strategy with the reduced-order model.
```

Finally, we visualize the snapshots to get a sense of how the solution looks qualitatively.

In [ ]:
def plot_heat_data(Z, title):
    """Visualize temperature data in space and time."""
    fig, ax = plt.subplots(1, 1)

    # Plot a few snapshots over the spatial domain.
    sample_columns = [0, 20, 80, 160, 320, 640]
    color = iter(plt.cm.viridis_r(np.linspace(0, 1, len(sample_columns))))

    for j in sample_columns:
        q_all = np.concatenate([[0], Z[:,j], [0]])  # Pad results with boundary conditions.
        ax.plot(x_all, q_all, color=next(color), label=f"q(x,t_{j})")

    ax.set_xlim(0, 1)
    ax.set_xlabel("x")
    ax.set_ylabel("q(x,t)")
    ax.legend(loc=(1.05, .05))
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    fig.suptitle(title)

In [ ]:
plot_heat_data(Q, "Snapshot Data")

Initially there is more heat toward the center of the rod, which then diffuses out of the ends of the rod.

At this point, we have gathered some training data by simulating the full-order model.
We also have an initial condition and space and time domains.

| Name | Symbol | Code Variable |
| :--- | :----: | :------------ |
| State snapshots | $\mathbf{Q}$ | `Q` |
| Initial state | $\mathbf{q}_0$ | `q0` |
| Spatial variable | $\Omega$ | `x` |
| Time domain | $[t_0,t_f]$ | `t` |

## Using the Package

The full-order model has the form {eq}`eq_fom`,

$$
    \frac{\text{d}}{\text{d}t}\mathbf{q}(t)
    = \mathbf{A}\mathbf{q}(t),\qquad\mathbf{q}(0)
    = \mathbf{q}_0,
$$

with $\mathbf{q}(t)\in\mathbb{R}^{n}$ and $\mathbf{A}\in\mathbb{R}^{n\times n}$.
We seek a reduced-order model with that same structure,

$$
    \frac{\text{d}}{\text{d}t}\widehat{\mathbf{q}}(t)
    = \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t),\qquad\widehat{\mathbf{q}}(0)
    = \widehat{\mathbf{q}}_0,
$$ (eq_rom)

but with $\widehat{\mathbf{q}}(t)\in \mathbb{R}^{r}$ and $\widehat{\mathbf{A}}\in\mathbb{R}^{r\times r}$ for some $r\ll n$.
Operator Inference constructs {eq}`eq_rom` by solving a data-driven minimization for $\widehat{\mathbf{A}}$,

$$
    \min_{\widehat{\mathbf{A}}\in\mathbb{R}^{r\times r}}\sum_{j=0}^{k-1}\left\|
        \widehat{\mathbf{A}}\mathbf{V}^{\top}\mathbf{q}_{j} - \mathbf{V}^{\top}\dot{\mathbf{q}}_{j}
    \right\|_{2}^2
$$ (eq_opinf)

where $\mathbf{V}\in\mathbb{R}^{n \times r}$ is a basis for $\mathbb{R}^{r}$ and $\dot{\mathbf{q}}_{j} = \frac{\text{d}}{\text{d}t}\mathbf{q}(t)\big|_{t = t_j}$ is a measurement of the time derivative of $\mathbf{q}(t)$ at time $t = t_{j}$.

We have several tasks to consider:
1. Choosing the reduced-model dimension $r$,
2. Constructing a low-dimensional subspace (computing $\mathbf{V}$),
3. Computing the time derivative matrix $\dot{\mathbf{Q}}$,
4. Constructing the reduced-order model {eq}`eq_rom` via Operator Inference {eq}`eq_opinf`,
5. Simulating the reduced-order model, and
6. Evaluating the performance of the reduced-order model.

We will do this all at once, then show each step in more detail.

In [ ]:
import rom_operator_inference as opinf

Vr, _ = opinf.pre.pod_basis(Q, r=2)                     # Construct the reduced basis.
Qdot = opinf.pre.ddt(Q, dt, order=6)                    # Calculate the time derivative matrix.
rom = opinf.ContinuousOpInfROM(modelform="A")           # Define the model structure.
rom.fit(Vr, Q, Qdot)                                    # Construct the ROM with Operator Inference.
Q_ROM = rom.predict(q0, t, method="BDF", max_step=dt)   # Simulate the ROM.
opinf.post.frobenius_error(Q, Q_ROM)[1]                 # Calculate the relative error of the ROM simulation.

## Choosing the Dimension of the Reduced-order Model

There are several ways to choose $r$ in an informed way.
A simple choice is to look at the singular values $\{\sigma_j\}_{j=1}^{n}$ of the snapshot matrix $\mathbf{Q}$ and select the number of $\sigma_{j}$ that are greater than a given threshold.
This also gives us a sense of whether or not we expect model reduction to be successful: if the singular values do not decay quickly, then we will need many modes to capture the behavior of the system.

In [ ]:
svdvals = la.svdvals(Q)
svdvals

In [ ]:
import inspect

def print_doc(func):
    print(f"def {func.__name__}{inspect.signature(func)}:",
          func.__doc__, sep="\n    ")

In [ ]:
import rom_operator_inference as opinf

print_doc(opinf.pre.svdval_decay)

In [ ]:
opinf.pre.svdval_decay(svdvals, 1e-4, plot=True)

We can also look at the relative contribution of the singular values, i.e., choose $r$ such that

$$
    \kappa_r = \frac{\sum_{j=1}^r \sigma_j^2}{\sum_{j=1}^n \sigma_j^2}
$$

is greater than a given value (usually something very close to $1$).

In [ ]:
print_doc(opinf.pre.cumulative_energy)

In [ ]:
r = opinf.pre.cumulative_energy(svdvals, .999999, plot=False)
print(f"r = {r}")

This indicates that we can capture 99.9999% of the behavior of the full-order snapshots with only 2 modes.
This is a very small choice of $r$, but it is also a very simple problem, so for now we select $r = 2$.

## Constructing a Low-dimensional Subspace

Next, we need a basis matrix $\mathbf{V}_{r}\in\mathbb{R}^{n \times r}$ to define the linear subspace to which the ROM states will be confined.
One of the most standard strategies, which aligns with our analysis of the singular values of $\mathbf{Q}$, is the _POD basis of rank $r$_ corresponding to $\mathbf{Q}$.
If $\mathbf{Q}$ has the singular value decomposition

$$
\mathbf{Q} = \boldsymbol{\Phi} \boldsymbol{\Sigma} \boldsymbol{\Psi}^{\top},
$$

then the POD basis of rank $r$ consists of the first $r$ columns of $\boldsymbol{\Phi}$, i.e., the dominant $r$ left singular vectors of $\mathbf{Q}$:

$$
\mathbf{V}_{r} := \boldsymbol{\Phi}_{:,:r}.
$$

In [ ]:
print_doc(opinf.pre.pod_basis)

In [ ]:
r = 2
Vr, _ = opinf.pre.pod_basis(Q, r, mode="dense")
print(f"Shape of Vr: {Vr.shape}")

To get a sense of the kinds of solutions we may see, we plot the columns of $\mathbf{V}_r$.
All solutions of the resulting ROM can only be linear combinations of these columns.

In [ ]:
for j in range(Vr.shape[1]):
    plt.plot(x_all, np.concatenate(([0], Vr[:,j], [0])), label=f"POD mode {j+1}")
plt.legend(loc="upper right")
plt.show()

## Estimating Time Derivatives

Operator Inference constructs a reduced-order model by solving a least-squares regression problem that corresponds to the form of the model.
In this case, the original model has the form $\frac{\text{d}}{\text{d}t}\mathbf{q}(t) = \mathbf{A}\mathbf{q}(t)$.
The snapshot matrix $\mathbf{Q}$ contains data for $\mathbf{q}(t)$, but we also need data for $\frac{\text{d}}{\text{d}t}\mathbf{q}(t)$.
In this simple example, we can directly compute the _snapshot time derivative matrix_ $\dot{\mathbf{Q}}\in\mathbb{R}^{n\times k}$ that corresponds to the snapshots by setting $\dot{\mathbf{Q}} = \mathbf{A} \mathbf{Q}$.

In [ ]:
Qdot = A @ Q

print(f"Shape of Q:\t{Q.shape}")
print(f"Shape of Qdot:\t{Qdot.shape}")

If the matrix $\mathbf{A}$ is unknown or computationally unavailable, the time derivative matrix can be estimated through finite differences of the snapshots.
The `pre` submodule has some convenience tools for this.
Since our time domain is uniformly spaced, we use `opinf.pre.ddt_uniform()`; for snapshots that are not uniformly spaced in time, see `opinf.pre.ddt_nonuniform()`.

In [ ]:
print_doc(opinf.pre.ddt_uniform)

In [ ]:
Qdot2 = opinf.pre.ddt_uniform(Q, dt, order=6)

# Check that the estimate is close to the true time derivatives.
la.norm(Qdot - Qdot2, ord=np.inf) / la.norm(Qdot, ord=np.inf)

```{note}
The finite difference approximation for $\dot{\mathbf{Q}}$ commutes with the projection to a low-dimensional subspace, that is, $\mathbf{V}_{r}^\top\frac{\text{d}}{\text{d}t}\left[\mathbf{Q}\right] = \frac{\text{d}}{\text{d}t}\left[\mathbf{V}_{r}^{\top}\mathbf{Q}\right]$.
To save memory, the snapshot matrix may be projected first, and the projected time derivatives can be calculated from the projected snapshots.
The ROM classes in the next section accept both full-order ($n \times k$) or reduced-order ($r\times k$) snapshot and time derivative matrices as training data.
```

In [ ]:
Q_ = Vr.T @ Q                                   # Project the state snapshots.
Qdot_ = opinf.pre.ddt_uniform(Q_, dt, order=6)  # Estimate the projected time derivatives.

np.allclose(Vr.T @ Qdot2, Qdot_)                # Same as project the full-order time derivatives.

## Constructing the ROM via Operator Inference

We now have training data and a linear basis for a low-dimensional subspace.

| Name | Symbol | Code Variable |
| :--- | :----: | :------------ |
| State snapshots | $\mathbf{Q}$ | `Q` |
| Time derivatives | $\dot{\mathbf{Q}}$ | `Qdot` |
| POD basis | $\mathbf{V}_{r}$ | `Vr` |
| Initial state | $\mathbf{q}_0$ | `q0` |
| | |
| Spatial domain | $\Omega$ | `x` |
| Time domain | $[t_0,t_f]$ | `t` |

Next, we initialize a `rom_operator_inference` "ROM" class and fit it to the data.
Since the problem is continuous (time-dependent), we use the `ContinuousOpInfROM` class.
The constructor takes a single parameter, `modelform`, that specifies the structure of the desired model.

| Character | Name | Reduced-order model term |
| :-------- | :--- | :------- |
| `c` | Constant |  $\widehat{\mathbf{c}}$ |
| `A` | Linear |  $\widehat{\mathbf{A}}\widehat{\mathbf{q}}(t)$ |
| `H` | Quadratic |  $\widehat{\mathbf{H}}\left(\widehat{\mathbf{q}}\otimes\widehat{\mathbf{q}}\right)(t)$ |
| `G` | Cubic |  $\widehat{\mathbf{G}}\left(\widehat{\mathbf{q}}\otimes\widehat{\mathbf{q}}\otimes\widehat{\mathbf{q}}\right)(t)$ |
| `B` | Input |  $\widehat{\mathbf{B}}\mathbf{u}(t)$ |

Since we seek a ROM of the form $\frac{\text{d}}{\text{d}t}\widehat{\mathbf{q}}(t) = \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t)$, we set `modelform="A"`.
If there were a constant term, $\frac{\text{d}}{\text{d}t}\widehat{\mathbf{q}}(t) = \widehat{\mathbf{c}} + \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t)$, we would use `modelform="cA"`, and so on.
Beware that with cubic terms ($\widehat{\mathbf{G}}$), the data matrix starts to get very large.

In [ ]:
rom = opinf.ContinuousOpInfROM("A")
print(rom)

We now fit the model to the data by solving the least-squares problem {eq}`eq_opinf`,
\begin{align*}
    \min_{\widehat{\mathbf{A}}\in\mathbb{R}^{r\times r}}\sum_{j=0}^{k-1}\left\|
        \widehat{\mathbf{A}}\mathbf{V}^{\top}\mathbf{q}_{j} - \mathbf{V}^{\top}\dot{\mathbf{q}}_{j}
    \right\|_{2}^2
    =
    \min_{\widehat{\mathbf{A}}\in\mathbb{R}^{r\times r}}\sum_{j=0}^{k-1}\left\|
        \widehat{\mathbf{A}}\widehat{\mathbf{q}}_{j} - \dot{\widehat{\mathbf{q}}}_{j}
    \right\|_{2}^2
    = \min_{\widehat{\mathbf{A}}\in\mathbb{R}^{r\times r}}\left\|
        \widehat{\mathbf{A}}\widehat{\mathbf{Q}} - \dot{\widehat{\mathbf{Q}}}
    \right\|_{F}^2,
\end{align*}
where
\begin{align*}
    \widehat{\mathbf{Q}} &= \mathbf{V}_r^{\top}\mathbf{Q},
    &
    \dot{\widehat{\mathbf{Q}}} &= \mathbf{V}_r^{\top}\dot{\mathbf{Q}}.
\end{align*}

This is all done in the `fit()` method, given $\mathbf{V}_r$, $\mathbf{Q}$, and $\dot{\mathbf{Q}}$.

In [ ]:
print_doc(rom.fit)

In [ ]:
rom.fit(Vr, Q, Qdot)

After fitting the model, we can directly examine the inferred operators of the model.

In [ ]:
rom.A_.entries

Because this is such a simple problem, Operator Inference recovers the exact same operator $\widehat{\mathbf{A}}$ as intrusive projection, i.e., $\tilde{\mathbf{A}} = \mathbf{V}_r^{\top} \mathbf{A} \mathbf{V}_r$:

In [ ]:
Atilde = Vr.T @ A @ Vr
Atilde

In [ ]:
np.allclose(rom.A_.entries, Atilde)

## Simulating the Learned ROM

Once the model is fit, we may simulate the ROM with the `predict()` method, which wraps `scipy.integrate.solve_ivp()`.
This method takes an initial condition from the original space $\mathbb{R}^n$, projects it to $\mathbb{R}^r$, simulates the ROM in $\mathbb{R}^r$, and maps the results to $\mathbb{R}^n$.

In [ ]:
print_doc(rom.predict)

In [ ]:
Q_ROM = rom.predict(q0, t, method="BDF", max_step=dt)
Q_ROM.shape

:::{note}

The `predict()` method is convenient, but `scipy.integrate.solve_ivp()` implements relatively few time integration schemes.  However, the ROM can be simulated by **any** ODE solver scheme by extracting the inferred operator $\widehat{\mathbf{A}}$. 
If `solver(A, q0)` were a solver for systems of the form $\frac{\text{d}}{\text{d}t}\mathbf{q} = \mathbf{A}\mathbf{q}(t),\ \mathbf{q}(0) = \mathbf{q}_0$, we could simulate the ROM with the following code.

```python
q0_ = Vr.T @ q0                      # Project the initial conditions.
Q_ = solver(rom.A_.entries, q0_)     # Solve the ROM in the reduced space.
Q_ROM = Vr @ Q_                      # Map the results to the full space.
```

More generally, the ROM object has a method `evaluate()` that represents the right-hand side of the model, the $\widehat{\mathbf{f}}$ of $\frac{\text{d}}{\text{d}t}\widehat{\mathbf{q}}(t) = \widehat{\mathbf{f}}(t, \widehat{\mathbf{q}}(t))$.
All-purpose integrators can therefore be applied to the function `rom.evaluate()`.
:::

## Evaluating ROM Performance

To see how the ROM does, we begin by visualizing the simulation output `Q_ROM`.
It should look similar to the plot of the snapshot data `Q`.

In [ ]:
plot_heat_data(Q_ROM, "ROM State Output")

For more detail, we evaluate the $\ell^2$ error of the ROM output in time, comparing it to the snapshot set.

In [ ]:
print_doc(opinf.post.lp_error)

In [ ]:
abs_l2err, rel_l2err = opinf.post.lp_error(Q, Q_ROM)
plt.semilogy(t, abs_l2err)
plt.title("Absolute Error")
plt.show()

In this **extremely** simple example, the error decreases with time (as solutions get quickly pushed to zero), but this is not the kind of error behavior that should be expected for less trivial systems.

We can also get a scalar error measurement by calculating the relative Frobenius norm error.

In [ ]:
print_doc(opinf.post.frobenius_error)

In [ ]:
abs_froerr, rel_froerr = opinf.post.frobenius_error(Q, Q_ROM)
print(f"Relative error: {rel_froerr:%}")

In other words, the ROM simulation is within 0.1% of the snapshot data.
Note that this value is very close to the projection error that we calculated earlier.